In [124]:
%matplotlib inline
import torch
import random
from d2l import torch as d2l

In [125]:
# 生产数据
def synethetic_data(w, b, num_examples):
    x = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(x, w) + b
    y = y + torch.normal(0, 0.01, y.shape)
    return x , y

In [126]:
# batch_size data
def data_iter(features, labels, batch_size):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i : min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]        

In [127]:
for x , y in data_iter(features, labels, 10):
    print(x.shape)
    print(y.shape)
    break

torch.Size([10, 2])
torch.Size([10, 1])


In [128]:
# 线性模型
def linreg(w, b, x):
    return torch.matmul(x, w) + b

In [129]:
# 损失函数
def squared_loss(y, y_hat):
    return (y - y_hat) ** 2 / 2

In [130]:
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [153]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
true_w = torch.tensor([[2], [-3.4]])
true_b = torch.tensor([4.2])
features, labels = synthetic_data(true_w, true_b, 1000)

lr = 1
num_epochs = 10
net = linreg
loss = squared_loss
batch_size = 10

for epoch in range(num_epochs):
    for x , y in data_iter(features, labels, batch_size):
        l = loss(net(w, b, x), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(w, b, features), labels)
        print('epoch {}, loss {}'.format(epoch, train_l.mean()))

epoch 0, loss 8.643382898299024e-05
epoch 1, loss 4.84566880913917e-05
epoch 2, loss 5.4561813158215955e-05
epoch 3, loss 6.973880954319611e-05
epoch 4, loss 7.778970757499337e-05
epoch 5, loss 6.0252037656027824e-05
epoch 6, loss 5.9761096053989604e-05
epoch 7, loss 5.2243303798604757e-05
epoch 8, loss 6.297496292972937e-05
epoch 9, loss 4.928092675982043e-05


In [151]:
print(f'w的估计误差: {true_w - w}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([[ 0.0026],
        [-0.0018]], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0002], grad_fn=<SubBackward0>)
